In [2]:
from utils import *
from models import *

from tqdm import tqdm
from pprint import pprint
import json
import _pickle as pkl
from time import time
from random import shuffle, seed

In [3]:
glove = setup_glove()
print(glove.vectors.size())
VOCAB_SIZE = glove.vectors.size()[0]
with open('../data/data.json', 'r') as f:
    data = json.load(f)

torch.Size([400004, 50])


In [ ]:
idx = 5
example_X = (data['X_train'][idx])
example_y = (data['y_train'][idx])
print("ID:", example_X[0])
print("Context:", example_X[1])
print("Question:", example_X[2])
print("Answer Span:", example_y)
print("Answer:", example_X[3])

_, p, X, y = make_data([example_X], [example_y], 1, glove)
print(y)
print(get_answer_span(y[0], p[0], example_X[1]))
print("===========")

seed(1)
zipped_data = list(zip(data['X_train'], data['y_train']))
shuffle(zipped_data)
data['X_train'], data['y_train'] = list(zip(*zipped_data))

num_train = 128
num_val = 32

idxs_train, padlens_train, X_train, y_train = make_data(data['X_train'], data['y_train'], num_train, glove)
idxs_val, padlens_val, X_val, y_val = make_data(data['X_val'], data['y_val'], num_val, glove)
print(len(X_train), len(y_train), len(X_val), len(y_val))

In [ ]:
conf = {"vocab": glove.vectors,
        "learning_rate": 0.01,
        "epochs": 10,
        "hidden_size": 50,
        "batch_size": 32,
        "opt": "Adamax",
        "n_layers": 1,
        "linear_dropout": 0.7,
        "seq_dropout": 0.7}
model = ModelV2(conf)
print(model)
if torch.cuda.is_available():
    model = model.cuda()
model_name = "%s_D%s_B%s_E%s_H%s_LR%s_O%s"%(type(model).__name__, num_train, model.batch_size, model.epochs, model.hidden_size, model.lr, conf["opt"])
print(model_name)

In [ ]:
tic = time()
v_preds, losses, vlosses = model.fit((X_train, y_train), (X_val, y_val))
toc = time()
print("took", toc-tic, "seconds")
torch.save(model, '../evaluation/models/%s'%model_name)
import matplotlib.pyplot as plt
plt.plot(list(range(len(losses))), losses, label='train')
plt.plot(list(range(len(vlosses))), vlosses, label='val')
plt.legend()
plt.show()

In [ ]:
# model = torch.load('../evaluation/models/%s'%model_name)
# model = torch.load('../evaluation/models/ModelV2_D512_B32_E5_H32_LR0.01_OAdamax')
print(model)

In [ ]:
bs = 64
print("batch size", bs)

num_test = 4096 # len(data['X_test'])
idxs_test, padlens_test, X_test, y_test = make_data(data['X_test'], data['y_test'], num_test, glove)

dev_results = {}

def switch_idxs(pred):
    if pred[0]>pred[1]:
        pred[0], pred[1] = pred[1], pred[0]
    return pred

print("Test data size:", num_test)
for bindex,  i in tqdm(enumerate(range(0, len(y_test)-bs+1, bs))):
    print("batch:", bindex)
    model.init_params(bs)
    Xb = torch.LongTensor(X_test[i:i+bs])
    yb = var(torch.LongTensor(y_test[i:i+bs]))
    if torch.cuda.is_available():
        Xb = Xb.cuda()
        yb = yb.cuda()
    pred = model.predict(Xb).data.tolist()
    pred = list(map(switch_idxs, pred))
    qids = [data['X_test'][j][0] for j in idxs_test[i:i+bs]]
    test_paras = [data['X_test'][j][1] for j in idxs_test[i:i+bs]]
    pads = padlens_test[i:i+bs]
    answers = list(map(get_answer_span, pred, pads, test_paras))
    batch_results = list(zip(qids, answers))
    # print(len(dict(batch_results)))
    dev_results.update(dict(batch_results))

In [ ]:
num_processed = len(dev_results)
print("processed:", num_processed)
dev_results['version'] = '1.1'
fname = 'run_%s_test%s.json'%(model_name, num_test)
with open('../data/%s'%fname, 'w') as f:
    print("saving to %s"%fname)
    json.dump(dev_results, f)